In [4]:
import pandas as pd
from ast import literal_eval
import csv
import pymssql
from datetime import datetime
import pyodbc

In [5]:
df = pd.read_csv('../data/movies.csv')

df = df[['id', 'title', 'release year', 'rating', 'Poster']]

movies = []

for i in range(len(df['release year'])):
    df.at[i, 'release year'] = ''.join(filter(str.isdigit, str(df.at[i, 'release year'])))

count = 0
for index, row in df.iterrows():
    count += 1
    year_date = datetime.strptime(row['release year'], '%Y').year
    movie = {"id": row['id'],'price_id': count, 'title': row['title'], 'release_year': year_date, 'rating': float(row['rating']), 'poster': row['Poster']}
    movies.append(movie)

In [6]:
# Connect to MSSQL database
#server = 'localhost'
server = 'MSI\SQLEXPRESSS'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

#connection = pymssql.connect(server=server, database=database, user=username, password=password)
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server+';DATABASE='+database+';UID='+username+';PWD=' + password)

# Insert dictionaries into the table
cursor = connection.cursor()
for dictionary in movies:
    cursor.execute("INSERT INTO movie (movie_id, price_id, title, release_year, rating, poster) VALUES (?, ?, ?, ?, ?, ?)", (dictionary['id'], dictionary['price_id'], dictionary['title'], dictionary['release_year'], dictionary['rating'], dictionary['poster']))

# Commit the changes and close the connection
connection.commit()
connection.close()